<h1> Réalisation d’un mini solveur de CSP binaires</h1>
Master MPRO parmentier-zribi

In [11]:
# -*- coding: utf-8 -*-
import time
import timeit
import random

In [12]:
variables=dict()
variables={}
variables["x1"]=[1,2]
variables["x2"]=[1,2]
contraintes=[[("x1","x2"),[(1,2),(2,2)]]]
instance=dict()
instance={}
#instance["x1"]=2
#instance["x2"]=1
#0: faux, 1: vérifié 2: incomplet


In [13]:
"""Checkconstraint sur une instance partielle"""
#Instance partielle: couple de valeurs pour x1, x2.
#{'x1':1,'x2':2}
#Pour tout checker, for loop sur les instances partielles, contraintes.
# ici constraint est de la forme [('x1', 'x2'), [(1, 2), (2, 2)]
# par ex contraintes[0]
def checkConstraint(constraint,instance):
    var1=constraint[0][0]
    var2=constraint[0][1]
    #print('var1 check: ',var1)
    #print('var2 check: ',var2)
    #print('instance keys ', instance.keys())
    if var1 in instance.keys() and var2 in instance.keys():
        for j in constraint[1]:
            if j[0]==instance[var1] and j[1]==instance[var2]:
                return(1)
        return(0)
    else:
        return(2)
        

print(checkConstraint(contraintes[0],{'x1':1,'x2':2}))

1


In [24]:
"Heuristics for variable selection in backtracking algorithm"
def strategy_selection_1(variables,instance):
    #The minimum index
     for x in variables.keys():
        if not(x in instance.keys()):
            return x
def strategy_selection_2(variables,instance):
    #The maximum index
     for x in sorted(list(variables.keys()),reverse=True):
        if not(x in instance.keys()):
            return x        
def strategy_selection_3(variables,instance):
    #Most constrained variable, min domain
    #Minimum remaining values variable first    
    keys_var_dict=list(variables.keys())
    values_list=list(variables.values())
    values_list_card=[len(i) for i in values_list]
    keys_var_dict_in_order=[keys_list for _,keys_list in sorted(zip(values_list_card,keys_list))]
    for x in keys_var_dict_in_order:
        if not(x in instance.keys()):
            return x
        
#pre-processing à faire avant de lancer le backtrack
def build_dict_var_number_constraints(variables,constraints):
    dict_var_number_constraints=dict()
    for var in variables.keys():
        dict_var_number_constraints[var]=0
    for var in variables.keys():
        for c in constraints:
            if var in c[0]:
                dict_var_number_constraints[var]+=1
    return dict_var_number_constraints

def strategy_selection_4(variables,instance,dict_var_number_constraints=False):
    #Most constraining variable
    #variable with most constraints on remaining variables
    #most edges in graph 
    #remaining_keys_variables=list(set(variables.keys())-set(instance.keys()))
    #build_dict_var_number_constraintsn pre-processing..
    if not dict_var_number_constraints:
        dict_var_number_constraints=build_dict_var_number_constraints(variables,constraints)
    elif isinstance(dict_var_number_constraints, dict):
        list_var_in_order=sorted(dict_var_number_constraints.items(),key=lambda t:t[1],reverse=True)
        list_var_in_order=[i[0] for i in list_var_in_order ]
        for x in list_var_in_order:
            if not(x in instance.keys()):
                return x
    else:
        return False
                
def strategy_selection_5(variables,instance,dict_var_number_constraints=False):
    #least constraining value heuristic
    if not dict_var_number_constraints:
        dict_var_number_constraints=build_dict_var_number_constraints(variables,constraints)
    elif isinstance(dict_var_number_constraints, dict):
        list_var_in_order=sorted(dict_var_number_constraints.items(),key=lambda t:t[1],reverse=False)
        list_var_in_order=[i[0] for i in list_var_in_order ]
        for x in list_var_in_order:
            if not(x in instance.keys()):
                return x
    else:
        return False

def strategy_selection_6(variables,instance):
    #Choose the variable with the minimum ratio domain size/number of constraints
    remaining_keys_variables=list(set(variables.keys())-set(instance.keys()))
    return random.choice(remaining_keys_variables)

#Pre-processing before backtrack if you use strategy_selection_7
def build_dict_var_ratio_domain_constraints(variables,constraints):
    #build the dict with the ratios domain size/number of constraints
    dict_ratios=dict()
    dict_var_number_constraints=build_dict_var_number_constraints(variables,constraints)
    #remaining_keys_variables=list(set(variables.keys())-set(instance.keys()))
    for var in variables.keys():
        if dict_var_number_constraints[var]!=0:
            dict_ratios[var]=len(variables[var])/dict_var_number_constraints[var]
        else:
            #100 is arbitrary. A big number
            dict_ratios[var]=len(variables[var])+100
    return dict_ratios

def strategy_selection_7(variables,instance,dict_ratios=False):
    #Choose a random variable among the remaining ones
    if not dict_ratios:
        dict_ratios=build_dict_var_ratio_domain_constraints(variables,constraints)
    elif isinstance(dict_ratios, dict):
        list_var_in_order=sorted(dict_ratios.items(),key=lambda t:t[1],reverse=False)
        list_var_in_order=[i[0] for i in list_var_in_order ]
        for x in list_var_in_order:
            if not(x in instance.keys()):
                return x
    else:
        return False

#Impact-based Search    

def selectVarAj(variables,instance,strategy_number,dict_var_number_constraints=False,dict_ratios=False):
    if strategy_number==1:
        return strategy_selection_1(variables,instance)
    elif strategy_number==2:
        return strategy_selection_2(variables,instance)
    elif strategy_number==3:
        return strategy_selection_3(variables,instance)
    elif strategy_number==4:
        if not dict_var_number_constraints:
            return strategy_selection_1(variables,instance)
        else:
            return strategy_selection_4(variables,instance,dict_var_number_constraints)
    elif strategy_number==5:
        if not dict_var_number_constraints:
            return strategy_selection_1(variables,instance)
        else:
            return strategy_selection_5(variables,instance,dict_var_number_constraints)
    elif strategy_number==6:
        return strategy_selection_6(variables,instance)
    elif strategy_number==7:
        if not dict_var_number_constraints:
            return strategy_selection_1(variables,instance)
        else:
            return strategy_selection_7(variables,instance,dict_ratios)

In [25]:
"""TEST"""
variables=dict()
variables={}
variables["x1"]=[1,2,3]
variables["x2"]=[1,2]
variables["x3"]=[1]
variables["x4"]=[1,2]
contraintes=[[("x1","x2"),[(1,2),(2,1),(3,1),(3,2)]],[("x2","x1"),[(1,2),(1,3),(2,1),(2,3)]],
             [("x2","x3"),[(2,1)]],[("x3","x2"),[(1,2)]],
            [("x1","x3"),[(2,1),(3,1)]],[("x3","x1"),[(1,2),(1,3)]],
            [("x1","x4"),[(1,2),(2,1),(3,1),(3,2)]],[("x4","x1"),[(1,2),(1,3),(2,1),(2,3)]],
            [("x3","x4"),[(1,2)]],[("x4","x3"),[(2,1)]],
            ]
#t1 = time.time()
#ac_3_bis(variables,contraintes)
#t2=time.time()
#duree=t2-t1

In [26]:
dict_var_const=build_dict_var_number_constraints(variables,contraintes)
print(dict_var_const)
list_order=sorted(dict_var_const.items(),key=lambda t:t[1],reverse=True)
print(list_order)
list_order=[i[0] for i in list_order]
print(list_order)
#print(OrderedDict(sorted(dict_var_const.items(), key=lambda t: t[1])))

{'x1': 6, 'x2': 4, 'x3': 6, 'x4': 4}
[('x1', 6), ('x3', 6), ('x2', 4), ('x4', 4)]
['x1', 'x3', 'x2', 'x4']


In [28]:
instance_partielle={"x1":3}
strategy_selection_1(variables,instance_partielle)

'x2'

In [ ]:
solutions=[]
def backTrack(instance,contraintes,exaustif=True):
    complet=True
    for c in contraintes:
        ch=checkConstraint(c,instance)
        if ch==0:
            #print(ch,False,"instance fausse !!!")
            return(False)
        if ch==2:
            #print("instance incomplete")
            complet=False
    if complet==True:
        print("instance complete")
        print("resultat:",instance)
        #solutions.append(instance)
        return(True)
    #On choisit une variable non instancée
    varAj=selectVarAj(variables,instance,1)
    #reordonner eventuellement varAj avant
    #Pour chaque valeur y de varAj dans son domaine
    for y in variables[varAj]:
        #print(varAj,"varAj ")
        #print(y,"y")
        #On ajoute les valeurs possibles de varAj
        instance2=instance.copy()
        #print(varAj)
        instance2[varAj]=y
        #print("instance actuelle",instance2)
        val=backTrack(instance2,contraintes)
        if val and exaustif==False:
            return(True)
    return(False)

In [ ]:
instance=dict()
variables={}
variables["x1"]=[1,2]
variables["x2"]=[1,2]
contraintes=[[("x1","x2"),[(1,2),(2,2)]]]
print(backTrack(instance,contraintes))

print(instance,"instance1")

print(variables)
#print(variables["x1"])

In [ ]:
instance

In [ ]:
contraintes

In [19]:
#Fonction séparée pour la selection du couple dans l'algo AC_3.
#Peut etre utile si existence d heuristiques interessantes
def selectCouple(aTester):
    #Different strategies can be chosen for the selection of the couple
    return aTester.pop()

In [ ]:
""""Fonction utilisee dans l'algo AC-3
Remarque: Ici, constraints n'est pas UNE contrainte mais l'ENSEMBLE des contraintes"""
def checkConstraint_ac3(constraints,instance):
    var1=list(instance.keys())[0]
    var2=list(instance.keys())[1]
    for c in constraints:
        if var1 in c[0] and var2 in c[0]:
            #si les nombres n'apparaissent meme pas dans les possibilites, on passe
            # a l iteration suivante
            """for i in range(len(c[1])):
                #check
                print("yes")
                print(var1,var2)
                print(c[1][i])
                print("-----")
                if instance[var1]==c[1][i][0] and instance[var2]==c[1][i][1]:
                    return True
                """
            if (instance[var1],instance[var2]) in c[1]:
                return True
    return False

Remarque: nouveau code ac_3, semble bien fonctionner. Testé sur l exemple du cours sur la coloration de graphe.

In [ ]:
"""
Algorithme AC-3
quand on supprime une valeur a la variable x,
on ne reexamine que les variables liees a x par une contrainte"""
def ac_3(variables,constraints):
    aTester=[]
    """Initialisation aTester"""
    for i in range(len(constraints)):
       # print('constraints[i]:',constraints[i])
        #on ajoute le couple (x,y)
        aTester.append(constraints[i][0])
        #print('Atester:',aTester)
    while len(aTester)!=0:
        """"Selection du couple """
        couple=selectCouple(aTester)
        list_values_without_support=[]
        #print('-*-*-*-*-*-*-')
        #print('new couple')
        #print('list no support',list_values_without_support)
        #pr  chaque valeur pour la variable x
        #print('couple:',couple)
        """"Parcourts des valeurs de x """
        for value_x in variables[couple[0]]:
            #print('nouvelle valeur de x',couple[0],value_x)
            support=False
            """"Parcourt des valeurs de y """
            for value_y in variables[couple[1]]:
                """"On verifie qu'il existe une contrainte Cx,y validee"""
                if checkConstraint_ac3(contraintes,{couple[0]:value_x,couple[1]:value_y}):
                    support=True
                    break;
            #print('support',support,couple[0],value_x,couple[1])
            """"On cree la liste des valeurs de x sans support"""
            if not support:
                #print('x', couple[0])
                list_values_without_support.append(value_x)
                #print('list no support',list_values_without_support)

        if len(list_values_without_support)!=0:
            #print('===')
            #print('reduce domaine of', couple[0], list_values_without_support)
            """"On met a jour le domaine de x """
            variables[couple[0]]=list(set(variables[couple[0]])-set(list_values_without_support))
            #print('variables: ',variables)
            """"On met a jour aTester"""
            for i in range(len(constraints)):
                if constraints[i][0][0]!=couple[1] and constraints[i][0][1]==couple[0]:
                    #print('couple étudié ', couple)
                    #print('ajout dans aTester',constraints[i][0])
                    aTester.append([constraints[i][0][0],constraints[i][0][1]])
                    #print('======')
            #print('======')
        #print('-*-*-*-*-*-*-')                              
    return variables
            

In [ ]:
variables=dict()
variables={}
variables["x1"]=[1,2,3]
variables["x2"]=[1,2]
variables["x3"]=[1]
contraintes=[[("x1","x2"),[(1,2),(2,1),(3,1),(3,2)]],[("x2","x1"),[(1,2),(1,3),(2,1),(2,3)]],
             [("x2","x3"),[(2,1)]],[("x3","x2"),[(1,2)]],
            [("x1","x3"),[(2,1),(3,1)]],[("x3","x1"),[(1,2),(1,3)]]
            ]
t1 = time.time()
ac_3(variables,contraintes)
t2=time.time()
duree=t2-t1

In [ ]:
variables

In [ ]:
duree

In [18]:
""""Fonction utilisee dans l'algo AC_3_Bis
Remarque: Ici, constraints n'est pas UNE contrainte mais l'ENSEMBLE des contraintes"""
def checkConstraint_ac3_bis(variables, constraints,var1,value_var1,var2):
    for c in constraints:
        if var1 in c[0] and var2 in c[0]:
            for value_var2 in variables[var2]:
                if (value_var1,value_var2) in c[1]:
                    return True
    return False

In [16]:
"""
Algorithme AC-3
quand on supprime une valeur a la variable x,
on ne reexamine que les variables liees a x par une contrainte"""
"""" Difference avec ac_3 ? au niveau de la fonction checkConstraint_ac3, ordre for c in constraints
et for value_y in domaines de y"""
""""Performances ac_3_bis peut etre legerement meilleur que ac_3"""
def ac_3_bis(variables,constraints):
    aTester=[]
    """Initialisation aTester"""
    for i in range(len(constraints)):
       # print('constraints[i]:',constraints[i])
        #on ajoute le couple (x,y)
        aTester.append(constraints[i][0])
    while len(aTester)!=0:
        """"Selection du couple """
        couple=selectCouple(aTester)
        list_values_without_support=[]
        #print('-*-*-*-*-*-*-')
        #print('new couple')
        #print('list no support',list_values_without_support)
        #pr  chaque valeur pour la variable x
        #print('couple:',couple)
        """"Parcourts des valeurs de x """
        for value_x in variables[couple[0]]:
            #print('nouvelle valeur de x',couple[0],value_x)
            support=False
            """"On verifie qu'il existe une contrainte Cx,y validee"""
            if checkConstraint_ac3_bis(variables,constraints,couple[0],value_x,couple[1]):
                support=True
                break;
            #print('support',support,couple[0],value_x,couple[1])
            """"On cree la liste des valeurs de x sans support"""
            if not support:
                #print('x', couple[0])
                list_values_without_support.append(value_x)
                #print('list no support',list_values_without_support)

        if len(list_values_without_support)!=0:
            #print('===')
            #print('reduce domaine of', couple[0], list_values_without_support)
            """"On met a jour le domaine de x """
            variables[couple[0]]=list(set(variables[couple[0]])-set(list_values_without_support))
            #print('variables: ',variables)
            """"On met a jour aTester"""
            for i in range(len(constraints)):
                if constraints[i][0][0]!=couple[1] and constraints[i][0][1]==couple[0]:
                    #print('couple étudié ', couple)
                    #print('ajout dans aTester',constraints[i][0])
                    aTester.append([constraints[i][0][0],constraints[i][0][1]])
                    #print('======')
            #print('======')
        #print('-*-*-*-*-*-*-')
       
                    
                                
    return variables
            

In [17]:
"""TEST"""
variables=dict()
variables={}
variables["x1"]=[1,2,3]
variables["x2"]=[1,2]
variables["x3"]=[1]
contraintes=[[("x1","x2"),[(1,2),(2,1),(3,1),(3,2)]],[("x2","x1"),[(1,2),(1,3),(2,1),(2,3)]],
             [("x2","x3"),[(2,1)]],[("x3","x2"),[(1,2)]],
            [("x1","x3"),[(2,1),(3,1)]],[("x3","x1"),[(1,2),(1,3)]]
            ]
t1 = time.time()
ac_3_bis(variables,contraintes)
t2=time.time()
duree=t2-t1


NameError: name 'selectCouple' is not defined

In [ ]:
variables

In [ ]:
duree

In [15]:
"""TEST"""
variables=dict()
variables={}
variables["x1"]=[1,2,3]
variables["x2"]=[1,2]
variables["x3"]=[1]
variables["x4"]=[1,2]
contraintes=[[("x1","x2"),[(1,2),(2,1),(3,1),(3,2)]],[("x2","x1"),[(1,2),(1,3),(2,1),(2,3)]],
             [("x2","x3"),[(2,1)]],[("x3","x2"),[(1,2)]],
            [("x1","x3"),[(2,1),(3,1)]],[("x3","x1"),[(1,2),(1,3)]],
            [("x1","x4"),[(1,2),(2,1),(3,1),(3,2)]],[("x4","x1"),[(1,2),(1,3),(2,1),(2,3)]],
            [("x3","x4"),[(1,2)]],[("x4","x3"),[(2,1)]],
            ]
t1 = time.time()
ac_3_bis(variables,contraintes)
t2=time.time()
duree=t2-t1

NameError: name 'ac_3_bis' is not defined

In [ ]:
variables

In [ ]:
duree

In [ ]:
"""Forward Checking
idéé: anticiper certaines prises de decisions dans le backtrack.
Dès qu’une variable x est instanciee à la valeur a, 
on filtre toutes les valeurs incompatibles avec〈x,a〉mais on ne propage pas plus.
Remarque: Ici, constraints n'est pas UNE contrainte mais l'ENSEMBLE des contraintes
var1 est la variable que l'on analyse.
"""
def forwardChecking(instance_partielle,variables,constraints,var1):
    for var2 in variables.keys():
        list_no_support=[]
        if var2 not in instance_partielle.keys():
            exist_constraint=False
            for c in constraints:
                if (var1,var2)==c[0]:
                    exist_constraint=True
                    break
            if exist_constraint:
                for value_var2 in variables[var2]:
                    #for loop supplementaire dans le cas où les contraintes sur x1,x2 ne sont 
                    #pas regroupees dans un seul bloc de constraints. A modifier si c est le cas
                    #finalement
                    for c in constraints:
                        if (var1,var2)==c[0] and not (instance_partielle[var1],value_var2) in c[1]:
                            list_no_support.append(value_var2)
        if len(list_no_support)!=0:
            variables[var2]=list(set(variables[var2])-set(list_no_support))
    return variables


In [ ]:
"""TEST"""
variables=dict()
variables={}
variables["x1"]=[1,2]
variables["x2"]=[1,2]
variables["x3"]=[1,3]
variables["x4"]=[1,2]
contraintes=[[("x1","x2"),[(1,2),(2,1),(3,1),(3,2)]],[("x2","x1"),[(1,2),(1,3),(2,1),(2,3)]],
             [("x2","x3"),[(2,1)]],[("x3","x2"),[(1,2)]],
            [("x1","x3"),[(2,1),(3,1)]],[("x3","x1"),[(1,2),(1,3)]],
            [("x1","x4"),[(1,2),(2,1),(3,1),(3,2)]],[("x4","x1"),[(1,2),(1,3),(2,1),(2,3)]],
            [("x3","x4"),[(1,2)]],[("x4","x3"),[(2,1)]],
            ]



In [ ]:
instance_partielle={"x1":3}

In [ ]:
forwardChecking(instance_partielle,variables,contraintes,"x1")

In [ ]:
variables

In [ ]:
def initAC4(variables,contraintes):
    Q=set()
    S=dict()
    S={}
    Count=dict()
    Count={}
    for c in contraintes:
        x=c[0][0]
        y=c[0][1]
        for a in variables[x]:
            total=0
            Count[(x,y,a)]=0
            for b in variables[y]:
                if (a,b) in c[1]:
                    total=total+1
                    if (y,b) in S.keys():
                        S[(y,b)].append((x,a))
                    else:
                        S[(y,b)]=[(x,a)]
                Count[(x,y,a)]=total
            if Count[(x,y,a)]==0:
                variables[x].remove(a)
                A=set([(x,a)])
                Q=Q|A
    return(Q,S,Count)
        

In [ ]:
def AC4(variables,contraintes):
    Q,S,Count=initAC4(variables,contraintes)
    #print("Q:",Q)
    #print("S:",S)
    #print("count:", Count)
    while len(Q)>0:
        yb=Q.pop()
        y,b=yb[0],yb[1]
        for xa in S[yb]:
            x,a=xa[0],xa[1]
            Count[(x,y,a)]=Count[(x,y,a)]-1
            if Count[(x,y,a)]==0 and a in variables[x]:
                varibales[x].remove(a)
                Q.remove((x,a))
    return(variables)

In [ ]:
"""TEST"""
variables=dict()
variables={}
variables["x1"]=[1,2]
variables["x2"]=[1,2]
variables["x3"]=[1,3]
variables["x4"]=[1,2]
contraintes=[[("x1","x2"),[(1,2),(2,1),(3,1),(3,2)]],[("x2","x1"),[(1,2),(1,3),(2,1),(2,3)]],
             [("x2","x3"),[(2,1)]],[("x3","x2"),[(1,2)]],
            [("x1","x3"),[(2,1),(3,1)]],[("x3","x1"),[(1,2),(1,3)]],
            [("x1","x4"),[(1,2),(2,1),(3,1),(3,2)]],[("x4","x1"),[(1,2),(1,3),(2,1),(2,3)]],
            [("x3","x4"),[(1,2)]],[("x4","x3"),[(2,1)]],
            ]
backTrack(instance,contraintes)


In [ ]:
print(variables)

AC4(variables,contraintes)

In [ ]:
def reines(n):
    variables=dict()
    variables={}
    contraintes=[]
    variables[1]=[j for j in range(1,(n+1)//2+1)]
    for i in range(2,n+1):
        variables[i]=[j for j in range(1,n+1)]
    for i in range(1,n+1):
        for j in range(i+1,n+1):
            c=[]
            for a in range(1,n+1):
                for b in range(1,n+1):
                    if a!=b and b-a!=j-i and a-b!=j-i:
                        c.append((a,b))
            contraintes.append([(i,j),c])
    return(variables,contraintes)

print(reines(5))
            

In [ ]:
variables,contraintes=reines(9)
instance=dict()
instance={}
print(backTrack(instance,contraintes))

In [ ]:
variables,contraintes=reines(5)
instance=dict()
instance={}

In [ ]:
AC4(variables,contraintes)

In [ ]:
variables,contraintes=reines(10,exhaustif=False)
instance=dict()
instance={}

variables=AC4(variables,contraintes)
print(variables)
print(backTrack(instance,contraintes))

In [ ]:
def colorGraph(nSommets,aretes,nCouleurs):
    variables=dict()
    variables={}
    contraintes=[]
    combinaisons=[]
    combinaisonsDiff=[]
    for c1 in range(nCouleurs):
        for c2 in range(nCouleurs):
            combinaisons.append((c1,c2))
            if c1!=c2:
                combinaisonsDiff.append((c1,c2))
    for s in range(nSommets):
        variables[s]=[k for k in range(nCouleurs)]
    for i in range(nSommets):
        for j in range(nSommets):
            if i!=j and ((i,j) in aretes or (j,i) in aretes):
                contraintes.append([(i,j),combinaisons.copy()])
            elif i!=j:
                contraintes.append([(i,j),combinaisonsDiff.copy()])
    return(variables,contraintes)

In [ ]:
print(colorGraph(6,[(1,2),(2,3),(1,3),(4,2),(4,5)],5))

In [ ]:
variables,contraintes=colorGraph(6,[(1,2),(2,3),(1,3),(4,2),(4,5)],3)
instance=dict()
instance={}

variables=AC4(variables,contraintes)
print(variables)
print(backTrack(instance,contraintes))

In [ ]:
import os

def readGraph(nomFichier):
    fichier=open(nomFichier,"r")
    l=fichier.readline()
    tab=l.split()
    nSommets=int(tab[2])
    aretes=[]
    for l in fichier.readlines():
        tab=l.split(' ')
        d=(int(tab[1]),int(tab[2][0]))
        if not(d in aretes):
            aretes.append(d)
    return(nSommets,aretes)

nSommetsG,aretesG=readGraph("out.col")

In [ ]:
nSommets,aretes=nSommetsG,aretesG
print(nSommets//2)

variables,contraintes=colorGraph(nSommets,aretes,nSommets//2)

#print(backTrack(instance,contraintes))

In [ ]:
aretesG